### Task B

In [201]:
import pandas as pd
import os
import datetime
import string
from nltk import FreqDist
from tqdm import tqdm
import spacy
import numpy as np
import openai
from openai.error import ServiceUnavailableError, APIError, Timeout
from sklearn.metrics.pairwise import cosine_similarity

current_path = os.getcwd()
current_path = current_path.split('Beer_Recommendation_Engine')
path = current_path[0] + 'Beer_Recommendation_Engine'
path

'/Users/drknt_rises/Desktop/Personal/Git Repos/Beer_Recommendation_Engine'

In [202]:
beer_reviews = pd.read_csv(path + '/Data/beer_reviews.csv')
beer_reviews = beer_reviews.drop_duplicates()

def process_date(data):
    from datetime import date

    o_date = str(data)
    o_date = o_date.split(' ')
    if o_date[1] == 'at':
        return date.today()
    else:
        return data


beer_reviews = beer_reviews.loc[~beer_reviews['comment'].isnull()]
beer_reviews['date'] = beer_reviews['date'].apply(lambda row: process_date(row))
beer_reviews['date'] = pd.to_datetime(beer_reviews['date'])
beer_reviews = beer_reviews.loc[beer_reviews['date'] >= datetime.datetime(year=2019, month=1, day=1)]
beer_reviews = beer_reviews.reset_index(drop=True)
beer_reviews = beer_reviews.drop_duplicates('comment')
beer_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11893 entries, 0 to 11963
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   pos           11893 non-null  int64         
 1   beer_name     11893 non-null  object        
 2   final_rating  11893 non-null  float64       
 3   looks         11893 non-null  float64       
 4   smell         11893 non-null  float64       
 5   taste         11893 non-null  float64       
 6   feel          11893 non-null  float64       
 7   overall       11893 non-null  float64       
 8   user          11893 non-null  object        
 9   place         11893 non-null  object        
 10  comment       11893 non-null  object        
 11  date          11893 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(1), object(4)
memory usage: 1.2+ MB


In [203]:
def generate_corpus(column, include_stopwords=True, stop_words=[]):
    corpus = " "
    if include_stopwords:
        for text in column:
            text = str(text)
            for i in list(string.punctuation):
                text = text.replace(i, ' ')
            
            text = text.lower()
            corpus += text
        
        corpus = corpus.split()
    else:
        for text in column:
            text = str(text)
            for i in list(string.punctuation):
                text = text.replace(i, ' ')
            
            text = text.lower()
            corpus += text
        
        corpus = corpus.split()

        corpus_filtered = []
        stop_words = set(stop_words)
        for word in corpus:
            if word not in stop_words:
                corpus_filtered.append(word)
        
        corpus = corpus_filtered
    
    return corpus

In [204]:
en = spacy.load("en_core_web_sm")
stop_words = en.Defaults.stop_words
corpus = generate_corpus(beer_reviews['comment'], include_stopwords=False, stop_words=stop_words)

word_freq = FreqDist()
for word in tqdm(corpus):
    word_freq[word] += 1

freq_dist = []
for rank, word in enumerate(word_freq):
    freq_dist.append([rank+1, word, word_freq[word]])

freq_dist = pd.DataFrame(freq_dist, columns=['Rank', 'Word', 'Count'])
freq_dist.to_csv(path + '/Support Files/Freq_Dist_new_comments.csv', index=False)
freq_dist

100%|██████████| 607954/607954 [00:00<00:00, 2566833.66it/s]


,Rank,Word,Count
0,1,beer,8692
1,2,head,6471
2,3,s,6144
3,4,taste,5652
4,5,chocolate,4784
...,...,...,...
22509,22510,to”,1
22510,22511,extend,1
22511,22512,canister,1
22512,22513,8000th,1


In [205]:
def process_column(column):
    data = list(column)
    for i in range(len(data)):
        text = str(data[i])
        for punct in list(string.punctuation):
             text = text.replace(punct, ' ')
        text = text.lower()
        data[i] = text
    return data

def replace_car_brands(data, map):
    for i in range(len(data)):
        text = data[i]
        l_text = text.split()
        for word in l_text:
            if word in map.keys():
                text = text.replace(word, map[word])
        data[i] = text.split()
    return data

In [206]:
comments = process_column(beer_reviews['comment'])

beer_attribute_map = pd.read_csv(path + '/Support FIles/Beeer_attributes.csv')

beer_attribute_dict = {}
for brand, model in zip(beer_attribute_map['Attribute'], beer_attribute_map['Variance']):
    beer_attribute_dict[model] = brand

comments = replace_car_brands(comments, beer_attribute_dict)

atribute_freq = []
for attribute in set(beer_attribute_dict.values()):
    try:
        atribute_freq.append([attribute, word_freq[attribute]])
    except:
        atribute_freq.append([attribute, 0])

atribute_freq = pd.DataFrame(atribute_freq, columns=['attribute', 'count'])
atribute_freq = atribute_freq.sort_values('count', ascending=False)
atribute_freq

,attribute,count
14,dark,4347
5,sweet,4075
6,coffee,3952
27,citrus,3059
19,vanilla,2952
25,balanced,1500
18,complex,984
13,cinnamon,908
17,fruity,843
11,hoppy,749


### Task C

In [207]:
attributes = list(set(beer_attribute_dict.values()))

def tf(text, attribute, map):
    text = str(text)
    text = text.strip()
    
    for punct in list(string.punctuation):
        text = text.replace(punct, ' ')
    text = text.lower()
    
    l_text = text.split()
    for word in l_text:
        if word in map.keys():
            text = text.replace(word, map[word])
    
    text = text.split()
    
    count = 0
    for t_word in text:
        if t_word==attribute:
            count += 1
    
    return count

for attribute in (pbar := tqdm(attributes)):
    pbar.set_description(f"Calculating TFs for {attribute}")
    beer_reviews[attribute] = beer_reviews['comment'].apply(lambda row: tf(row, attribute, beer_attribute_dict))
beer_reviews

Calculating TFs for malty: 100%|██████████| 30/30 [00:06<00:00,  4.44it/s]     


,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,apricot,cocoa,crisp,papaya,honey,balanced,vintage,citrus,silky,malty
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0,0,0,0,0,0,0,0,0,0
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0,0,0,0,0,0,0,0,0,0
2,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,alexnblanton,Arkansas,...,0,0,0,0,0,0,0,0,0,0
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0,0,0,0,0,0,0,0,0,0
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11959,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0,0,0,0,0,0,0,0,0,0
11960,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,0,0,0,0,0,0,0,0,0,0
11961,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,4,4,0,0,4,0,1,0,0,1
11962,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0,0,0,0,0,0,0,0,0,0


In [208]:
def idf(column, attribute, map):
    texts = list(column)
    n = len(texts)
    count = 0
    for i in range(n):
        text = texts[i]
        text = str(text)
        text = text.strip()
        
        for punct in list(string.punctuation):
            text = text.replace(punct, ' ')
        text = text.lower()
        
        l_text = text.split()
        for word in l_text:
            if word in map.keys():
                text = text.replace(word, map[word])
        
        text = text.split()

        if attribute in text:
            count += 1
        
    idf = np.log10(n / count)
    return idf

dict_idf = {}
for attribute in (pbar := tqdm(attributes)):
    dict_idf[attribute] = idf(beer_reviews['comment'], attribute, beer_attribute_dict)

dict_idf

100%|██████████| 30/30 [00:06<00:00,  4.61it/s]


{'herbal': 1.5129985544268276,
 'tangerine': 1.3217083599903958,
 'cherry': 1.5412653128271674,
 'roast/toast': 1.2031351461350095,
 'aggressive': 1.837245315754507,
 'sweet': 0.6115476976362433,
 'coffee': 0.768223468222004,
 'estery': 2.140792967639735,
 'spicy': 1.5687863864784304,
 'diacetyl': 2.6950801771716963,
 'robust': 1.8992001598276211,
 'hoppy': 1.300045159143066,
 'chocolaty': 0.6457006166600009,
 'cinnamon': 1.3949559054687393,
 'dark': 0.665358295552008,
 'melon': 1.436802161928665,
 'tart': 1.398597809258436,
 'fruity': 0.949160011621318,
 'complex': 1.0284062280455923,
 'vanilla': 0.8114187420180788,
 'apricot': 1.558095520933328,
 'cocoa': 1.4259565601711606,
 'crisp': 1.4191932168704706,
 'papaya': 1.5374723238100283,
 'honey': 1.61740952214931,
 'balanced': 0.7641735762207967,
 'vintage': 1.3380987761785652,
 'citrus': 0.7233019634476703,
 'silky': 1.539997298840532,
 'malty': 0.9297842474736399}

In [209]:
for attribute in attributes:
    beer_reviews[attribute + '_tf-idf'] = (beer_reviews[attribute] * dict_idf[attribute])
beer_reviews

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,apricot_tf-idf,cocoa_tf-idf,crisp_tf-idf,papaya_tf-idf,honey_tf-idf,balanced_tf-idf,vintage_tf-idf,citrus_tf-idf,silky_tf-idf,malty_tf-idf
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
2,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,alexnblanton,Arkansas,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0.000000,1.425957,0.0,0.0,0.000000,0.0,1.338099,0.0,0.0,0.929784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11959,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
11960,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
11961,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,6.232382,5.703826,0.0,0.0,6.469638,0.0,1.338099,0.0,0.0,0.929784
11962,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000


In [210]:
attributes = input(f"Enter three attributes seperated by comma and without space from the list: {set(beer_attribute_dict.values())}")
attributes = attributes.split(',')
passer = attributes.copy()

def similarity(data, vect_2):
    vect_1 = []
    for value in data:
        vect_1.append(value)
    
    sum1 = 0
    sum2 = 0
    sum3 = 0
    for i, j in zip(vect_1, vect_2):
        sum1 = sum1 + (i * j)
        sum2 = sum2 + (i * i)
        sum3 = sum3 + (j * j)
    
    if sum1==0 or sum2==0 or sum==0:
        similarity_score = 0
    else:
        similarity_score = sum1 / (np.sqrt(sum2) * np.sqrt(sum3))
    # print(vect_1, vect_2, similarity_score)
    return similarity_score
    
vect = []
for i, attribute in enumerate(attributes):
    vect.append(dict_idf[attribute])
    attributes[i] = attribute + '_tf-idf'

beer_reviews['similarity'] = beer_reviews[attributes].apply(tuple, axis=1)
beer_reviews['similarity'] = beer_reviews['similarity'].apply(lambda row: similarity(row, vect))
beer_reviews[['beer_name', 'comment', 'similarity']]

,beer_name,comment,similarity
0,Kentucky Brunch Brand Stout,"Amazing stout, expensive but worth the price!",0.000000
1,Kentucky Brunch Brand Stout,This beer meet and exceeded all the unreal hyp...,0.000000
2,Kentucky Brunch Brand Stout,Let it sit and warm to the room a little ... u...,0.000000
3,Kentucky Brunch Brand Stout,"A small pour, ebony dark, no real head. Intens...",0.384317
4,Kentucky Brunch Brand Stout,"Vintage 2022, served on tap at Toppling Goliat...",0.537051
...,...,...,...
11959,Samuel Adams Utopias,It seems weird to rate this like it's a normal...,0.000000
11960,Samuel Adams Utopias,"Shared 2017 canister. Fabulous, very complex a...",0.000000
11961,Samuel Adams Utopias,8000th review! 8 ounce pour from a 24 ounce bo...,0.539937
11962,Samuel Adams Utopias,Such a huge beer and a joy to drink. This is d...,0.000000


### Part D

In [211]:
re_calculate = False
if re_calculate == True:
    def sentiment_openai(data):
        openai_key = input("Provide API Key:")
        i = 0
        for iter in tqdm(data):
            sents = []
            final_rating, text = iter
            openai.api_key = openai_key
            prompt = f'''Provide a sentiment score for "{text}" in a range of +5 to -5. This comment was overall rated as {final_rating} by the user on a scale of 0-5. Watchout for the light human sarcasm in the quoted text. Only provide the number.'''

            try:
                # @backoff.on_exception(backoff.expo, (ServiceUnavailableError, APIError))
                def chat_completion(prompt):
                    response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                            {"role": "system", "content": prompt},
                        ]
                    )

                    score = response['choices'][0]['message']['content']
                    return score
                
                score = chat_completion(prompt)
            except (ServiceUnavailableError, APIError, Timeout) as error:
                score = -1
            sents.append([text, score])
            
            sents = pd.DataFrame(sents, columns=['comment', 'sentiment'])
            
            if i==0:
                sents.to_csv(path + '/Support Files/sentiments.csv', mode='a', index=False, header=True)
            else:
                sents.to_csv(path + '/Support Files/sentiments.csv', mode='a', index=False, header=False)
            
            i += 1
            # time.sleep(1)
        
    beer_reviews['sentiment'] = beer_reviews[['final_rating', 'comment']].apply(tuple, axis=1)
    sentiment_openai(beer_reviews['sentiment'])
    beer_reviews = beer_reviews.drop(['sentiment'], axis=1)

    sentiment = pd.read_csv(path + '/Support Files/sentiments.csv')
    sentiment = sentiment.astype({'sentiment':float})
    sentiment = sentiment.drop_duplicates('comment', keep='last')
    beer_reviews = beer_reviews.merge(sentiment, left_on='comment', right_on='comment', how='left')
else:
    sentiment = pd.read_csv(path + '/Support Files/sentiments.csv')
    sentiment = sentiment.astype({'sentiment':float})
    sentiment = sentiment.drop_duplicates('comment', keep='last')
    beer_reviews = beer_reviews.merge(sentiment, left_on='comment', right_on='comment', how='left')

beer_reviews['sentiment'] = beer_reviews['sentiment'].replace(0, 0.0000001)
display(beer_reviews)

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,crisp_tf-idf,papaya_tf-idf,honey_tf-idf,balanced_tf-idf,vintage_tf-idf,citrus_tf-idf,silky_tf-idf,malty_tf-idf,similarity,sentiment
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.00
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.00
2,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,alexnblanton,Arkansas,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,-3.00
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.384317,3.50
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0.0,0.0,0.000000,0.0,1.338099,0.0,0.0,0.929784,0.537051,4.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11888,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.76
11889,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.49
11890,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,0.0,0.0,6.469638,0.0,1.338099,0.0,0.0,0.929784,0.539937,3.89
11891,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.63


### Task E

In [212]:
max_sty = max(beer_reviews['similarity'])
min_sty = min(beer_reviews['similarity'])
df_bow = beer_reviews
recommended = beer_reviews.loc[(beer_reviews['sentiment'] > 0)].copy()
recommended['similarity_non_scaled'] = recommended['similarity']
recommended = recommended.loc[recommended['similarity_non_scaled'] > 0]
recommended['similarity'] = 5 * ((recommended['similarity'] - min_sty) / (max_sty - min_sty))
recommended['similarity'] = np.log(recommended['similarity']) / np.log(2.5)
# recommended = recommended.loc[recommended['similarity'] > 1]
recommended['sentiment'] = np.log(recommended['sentiment']) / np.log(2)
recommended['final_score'] = recommended['sentiment'] * recommended['sentiment']
recommended

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,honey_tf-idf,balanced_tf-idf,vintage_tf-idf,citrus_tf-idf,silky_tf-idf,malty_tf-idf,similarity,sentiment,similarity_non_scaled,final_score
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.712819,1.807355,0.384317,3.266532
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0.000000,0.000000,1.338099,0.0,0.0,0.929784,1.078016,2.263034,0.537051,5.121325
6,1,Kentucky Brunch Brand Stout,3.79,4.00,3.75,3.75,4.00,3.75,Emichaelbray,Pennsylvania,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.712819,1.321928,0.384317,1.747494
13,1,Kentucky Brunch Brand Stout,4.23,4.25,4.25,4.25,4.00,4.25,M-Fox24,New Jersey,...,0.000000,0.764174,0.000000,0.0,0.0,0.000000,0.712819,0.000000,0.384317,0.000000
24,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,TurboGP,Minnesota,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.712819,2.321928,0.384317,5.391350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11875,250,Samuel Adams Utopias,4.99,4.75,5.00,5.00,5.00,5.00,WormGod,New York,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.712819,2.319040,0.384317,5.377946
11880,250,Samuel Adams Utopias,4.55,4.50,4.50,4.50,4.50,4.75,VABA,Virginia,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.712819,2.169925,0.384317,4.708575
11882,250,Samuel Adams Utopias,4.56,4.50,4.00,4.75,4.75,4.75,LifesAnesthesia,Virginia,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.712819,2.169925,0.384317,4.708575
11890,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,6.469638,0.000000,1.338099,0.0,0.0,0.929784,1.083865,1.959770,0.539937,3.840699


In [213]:
recommended = recommended.loc[recommended['final_score'] > 1]
recommended = recommended.groupby('beer_name').agg(
                    avg_final_score = pd.NamedAgg('final_score', aggfunc='mean'),
                    count_reviews = pd.NamedAgg('beer_name', aggfunc='count')
                    )
recommended = recommended.reset_index(drop=False)
recommended['scaled_score'] = np.log10(recommended['count_reviews']) * np.log2(recommended['avg_final_score'])
recommended = recommended.sort_values('scaled_score', ascending=False)
recommended = recommended[:3]
print(f'Selected attributes: {passer}')
print('Three recommended beers:')
display(recommended)

Selected attributes: ['hoppy', 'malty', 'dark']
Three recommended beers:


,beer_name,avg_final_score,count_reviews,scaled_score
175,Pliny The Elder,12.140080,69,6.622994
209,Trappistes Rochefort 10,10.689706,86,6.612406
68,Dinner,21.444831,31,6.595634


### Part F

In [214]:
attributes = passer

re_calculate = False
if re_calculate:
    def spacySimilarity(comments, attributes, map):
        nlp = spacy.load("en_core_web_md")

        comments = list(comments)
        attributes = list(attributes)
        similarity_scores = []
        for comment in tqdm(comments):
            row = []
            row.append(comment)
            for attribute in attributes:
                text1 = attribute
                text2 = comment

                text2 = text2.strip()
                for punct in list(string.punctuation):
                    text2 = text2.replace(punct, ' ')
                text2 = text2.lower()

                l_text = text2.split()
                for word in l_text:
                    if word in map.keys():
                        text2 = text2.replace(word, map[word])

                text2 = ''.join(text2)

                doc1 = nlp(text1)
                doc2 = nlp(text2)
                similarity = doc1.similarity(doc2)
                row.append(similarity)
            similarity_scores.append(row)
        
        new_attributtes = ['comment']
        for i in range(len(attributes)):
            new_attributtes.append(attributes[i] + "_SSimilarity")
        
        similarity = pd.DataFrame(similarity_scores, columns=new_attributtes)
        similarity.to_csv(path + '/Support Files/similarity_spacy.csv', index=False)

    spacySimilarity(beer_reviews['comment'], set(beer_attribute_dict.values()), beer_attribute_dict)
    
    new_attributtes = ['comment']
    for i in range(len(attributes)):
        new_attributtes.append(attributes[i] + "_SSimilarity")
    
    similarity = pd.read_csv(path + '/Support Files/similarity_spacy.csv').drop_duplicates()
    similarity = similarity[new_attributtes]
    beer_reviews = beer_reviews.merge(similarity, left_on='comment', right_on='comment', how='left')
    display(beer_reviews)
else:
    new_attributtes = ['comment']
    for i in range(len(attributes)):
        new_attributtes.append(attributes[i] + "_SSimilarity")
    
    similarity = pd.read_csv(path + '/Support Files/similarity_spacy.csv').drop_duplicates()
    similarity = similarity[new_attributtes]
    beer_reviews = beer_reviews.merge(similarity, left_on='comment', right_on='comment', how='left')
    display(beer_reviews)

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,balanced_tf-idf,vintage_tf-idf,citrus_tf-idf,silky_tf-idf,malty_tf-idf,similarity,sentiment,hoppy_SSimilarity,malty_SSimilarity,dark_SSimilarity
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.00,0.195580,0.431294,0.276006
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.00,0.236214,0.447167,0.306804
2,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,alexnblanton,Arkansas,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,-3.00,0.204870,0.387106,0.366701
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0.0,0.000000,0.0,0.0,0.000000,0.384317,3.50,0.291125,0.519748,0.419661
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0.0,1.338099,0.0,0.0,0.929784,0.537051,4.80,0.225860,0.482156,0.401839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11888,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.76,0.271909,0.472258,0.335410
11889,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.49,0.222217,0.563357,0.361008
11890,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,0.0,1.338099,0.0,0.0,0.929784,0.539937,3.89,0.416497,0.677412,0.464622
11891,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,4.63,0.268542,0.440867,0.260693


In [215]:
new_attributtes = new_attributtes[1:]
beer_reviews['Ssimilarity'] = 1
for attri in new_attributtes:
    beer_reviews['Ssimilarity'] = beer_reviews['Ssimilarity'] * beer_reviews[attri]

max_sty = max(beer_reviews['Ssimilarity'])
min_sty = min(beer_reviews['Ssimilarity'])
df_wv = beer_reviews
recommended = beer_reviews.loc[(beer_reviews['sentiment'] > 0)].copy()
recommended['Ssimilarity_non_scaled'] = recommended['Ssimilarity']
recommended = recommended.loc[recommended['Ssimilarity_non_scaled'] > 0]
recommended['Ssimilarity'] = 5 * ((recommended['Ssimilarity'] - min_sty) / (max_sty - min_sty))
recommended['Ssimilarity'] = np.log(recommended['Ssimilarity']) / np.log(2.5)
# recommended = recommended.loc[recommended['Ssimilarity'] > 1]
recommended['sentiment'] = np.log(recommended['sentiment']) / np.log(2)
recommended['Sffinal_score'] = recommended['Ssimilarity'] * recommended['sentiment']
recommended

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,silky_tf-idf,malty_tf-idf,similarity,sentiment,hoppy_SSimilarity,malty_SSimilarity,dark_SSimilarity,Ssimilarity,Ssimilarity_non_scaled,Sffinal_score
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0.0,0.000000,0.000000,2.000000,0.195580,0.431294,0.276006,-0.728108,0.023282,-1.456215
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0.0,0.000000,0.000000,2.000000,0.236214,0.447167,0.306804,-0.394169,0.032407,-0.788339
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0.0,0.000000,0.384317,1.807355,0.291125,0.519748,0.419661,0.305294,0.063500,0.551775
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0.0,0.929784,0.537051,2.263034,0.225860,0.482156,0.401839,-0.084598,0.043760,-0.191449
5,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,smokinjimmyv,Illinois,...,0.0,0.000000,0.000000,2.169925,0.239741,0.471598,0.364591,-0.146606,0.041221,-0.318123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11888,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0.0,0.000000,0.000000,2.250962,0.271909,0.472258,0.335410,-0.101096,0.043070,-0.227563
11889,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,0.0,0.000000,0.000000,2.166715,0.222217,0.563357,0.361008,-0.051083,0.045194,-0.110682
11890,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,0.0,0.929784,0.539937,1.959770,0.416497,0.677412,0.464622,1.077264,0.131088,2.111190
11891,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0.0,0.000000,0.000000,2.211012,0.268542,0.440867,0.260693,-0.443936,0.030864,-0.981548


In [216]:
recommended = recommended.loc[recommended['Sffinal_score'] > 1]
recommended = recommended.groupby('beer_name').agg(
                    avg_final_score = pd.NamedAgg('Sffinal_score', aggfunc='mean'),
                    count_reviews = pd.NamedAgg('beer_name', aggfunc='count')
                    )
recommended  = recommended.reset_index(drop=False)
recommended['scaled_score'] = np.log10(recommended['count_reviews']) * np.log2(recommended['avg_final_score'])
recommended = recommended.sort_values('scaled_score', ascending=False)
recommended = recommended[:3]
print(f'Selected attributes: {passer}')
print('Three recommended beers:')
display(recommended)

Selected attributes: ['hoppy', 'malty', 'dark']
Three recommended beers:


,beer_name,avg_final_score,count_reviews,scaled_score
38,Bourbon County Brand Coffee Stout,7.683308,13,3.276918
34,Blessed,10.204765,9,3.197830
65,Darkstar November,12.513329,7,3.080715


In [225]:
'''beer_name	avg_final_score	count_reviews	scaled_score
175	Pliny The Elder	12.140080	69	6.622994
209	Trappistes Rochefort 10	10.689706	86	6.612406
68	Dinner	21.444831	31	6.595634'''

display(df_bow.loc[df_bow['beer_name'] == 'Pliny The Elder'][['beer_name', 'hoppy', 'malty', 'dark', 'similarity', 'sentiment']])
display(df_bow.loc[df_bow['beer_name'] == 'Trappistes Rochefort 10'][['beer_name', 'hoppy', 'malty', 'dark', 'similarity', 'sentiment']])
display(df_bow.loc[df_bow['beer_name'] == 'Dinner'][['beer_name', 'hoppy', 'malty', 'dark', 'similarity', 'sentiment']])

,beer_name,hoppy,malty,dark,similarity,sentiment
2284,Pliny The Elder,0,0,0,0.0,4.23
2285,Pliny The Elder,0,0,0,0.0,4.25
2286,Pliny The Elder,0,0,0,0.0,4.50
2287,Pliny The Elder,0,0,0,0.0,4.62
2288,Pliny The Elder,0,0,0,0.0,4.44
...,...,...,...,...,...,...
2520,Pliny The Elder,0,0,0,0.0,4.50
2521,Pliny The Elder,0,0,0,0.0,4.00
2522,Pliny The Elder,0,0,0,0.0,4.43
2523,Pliny The Elder,0,0,0,0.0,4.50


,beer_name,hoppy,malty,dark,similarity,sentiment
8760,Trappistes Rochefort 10,0,0,3,0.384317,4.79
8761,Trappistes Rochefort 10,0,1,0,0.537051,4.31
8762,Trappistes Rochefort 10,0,0,1,0.384317,5.00
8763,Trappistes Rochefort 10,0,0,0,0.000000,4.50
8764,Trappistes Rochefort 10,0,0,0,0.000000,3.20
...,...,...,...,...,...,...
8912,Trappistes Rochefort 10,0,0,0,0.000000,4.00
8913,Trappistes Rochefort 10,0,0,1,0.384317,4.70
8914,Trappistes Rochefort 10,0,0,0,0.000000,-1.50
8915,Trappistes Rochefort 10,0,0,1,0.384317,1.75


,beer_name,hoppy,malty,dark,similarity,sentiment
3163,Dinner,0,0,0,0.000000,3.93
3164,Dinner,0,1,0,0.537051,4.35
3165,Dinner,0,1,0,0.537051,4.84
3166,Dinner,1,0,0,0.750917,4.72
3167,Dinner,0,1,0,0.537051,-2.50
...,...,...,...,...,...,...
3260,Dinner,0,0,0,0.000000,4.50
3261,Dinner,0,0,0,0.000000,4.93
3262,Dinner,0,0,0,0.000000,4.71
3263,Dinner,0,0,0,0.000000,4.50


In [224]:
'''beer_name	avg_final_score	count_reviews	scaled_score
38	Bourbon County Brand Coffee Stout	7.683308	13	3.276918
34	Blessed	10.204765	9	3.197830
65	Darkstar November	12.513329	7	3.080715'''

display(df_wv.loc[df_wv['beer_name'] == 'Bourbon County Brand Coffee Stout'][['beer_name', 'hoppy', 'malty', 'dark', 'Ssimilarity', 'sentiment']])
display(df_wv.loc[df_wv['beer_name'] == 'Blessed'][['beer_name', 'hoppy', 'malty', 'dark', 'Ssimilarity', 'sentiment']])
display(df_wv.loc[df_wv['beer_name'] == 'Darkstar November'][['beer_name', 'hoppy', 'malty', 'dark', 'Ssimilarity', 'sentiment']])

,beer_name,hoppy,malty,dark,Ssimilarity,sentiment
1378,Bourbon County Brand Coffee Stout,0,0,2,0.036627,2.50
1379,Bourbon County Brand Coffee Stout,0,1,2,0.074009,4.50
1380,Bourbon County Brand Coffee Stout,0,1,3,0.175873,4.24
1381,Bourbon County Brand Coffee Stout,0,1,1,0.041365,4.80
1382,Bourbon County Brand Coffee Stout,0,0,0,0.056482,4.89
...,...,...,...,...,...,...
1445,Bourbon County Brand Coffee Stout,0,0,0,0.063043,3.50
1446,Bourbon County Brand Coffee Stout,0,0,1,0.081039,4.50
1447,Bourbon County Brand Coffee Stout,0,0,5,0.058021,4.50
1448,Bourbon County Brand Coffee Stout,0,0,0,0.014484,4.50


,beer_name,hoppy,malty,dark,Ssimilarity,sentiment
187,Blessed,0,0,0,0.033381,-3.000000e+00
188,Blessed,0,0,3,0.047081,3.500000e+00
189,Blessed,0,0,0,0.045532,4.500000e+00
190,Blessed,0,0,0,0.000998,5.000000e+00
191,Blessed,0,0,0,0.000258,1.000000e-07
...,...,...,...,...,...,...
260,Blessed,0,0,1,0.070980,4.680000e+00
261,Blessed,0,1,1,0.066944,-2.750000e+00
262,Blessed,0,0,0,0.070922,4.400000e+00
263,Blessed,0,0,1,0.079347,3.800000e+00


,beer_name,hoppy,malty,dark,Ssimilarity,sentiment
11320,Darkstar November,0,0,0,0.060647,3.900000e+00
11321,Darkstar November,0,1,0,0.016110,4.500000e+00
11322,Darkstar November,0,1,1,0.054936,4.350000e+00
11323,Darkstar November,0,0,1,0.053288,4.850000e+00
11324,Darkstar November,0,1,0,0.046760,4.300000e+00
11325,Darkstar November,0,0,3,0.044397,3.750000e+00
11326,Darkstar November,0,0,0,0.045871,3.700000e+00
11327,Darkstar November,0,1,0,0.066568,4.100000e+00
11328,Darkstar November,0,0,0,0.004089,5.000000e+00
11329,Darkstar November,0,1,1,0.040357,3.500000e+00


Task G

> Enter here

### Task H

In [219]:
def lift(texts, x, y, words=float('inf')):
    x = x.replace(' ', '')
    y = y.replace(' ', '')

    n = len(texts)
    count_x = 0
    count_y = 0
    count_x_y = 0

    for i in range(n):
        x_indices = []
        y_indices = []
        text = texts[i].split()
        
        for j in range(len(text)):
            if x == text[j]:
                x_indices.append(j)
            elif y == text[j]:
                y_indices.append(j)

        n_words = []    
        for x_index in x_indices:
            for y_index in y_indices:
                n_words.append(abs(x_index - y_index) - 1)
        
        # print(len(n_words) > 0, len(x_indices) > 0, len(y_indices) > 0)
        if len(n_words) > 0:
            if float(min(n_words)) <= float(words):
                count_x_y += 1
        if len(x_indices) > 0:
            count_x += 1
        if len(y_indices) > 0:
            count_y += 1
        
    # print(n, x, y, count_x_y, count_x, count_y)
    
    try:
        lift = (n * count_x_y) / (count_x * count_y)
    except ZeroDivisionError:
        lift = np.nan
    return lift

In [220]:
comments = process_column(beer_reviews['comment'])
beer_name = list(beer_reviews['beer_name'])
for i in range(len(beer_name)):
    beer_name[i] = beer_name[i].replace(' ', '')

comments = replace_car_brands(comments, beer_attribute_dict)
for i in range(len(comments)):
    comments[i] = ' '.join(comments[i])
    comments[i] = beer_name[i] + ' - ' + comments[i]
    comments[i] = comments[i].strip()

beers = list(beer_reviews['beer_name'].value_counts()[:10].index)
attributes = ['hoppy', 'malty', 'dark', 'complex']

lifts = []
for beer in beers:
    row = []
    row.append(beer)
    for attribute in attributes:
        lift_score = lift(comments, beer, attribute)
        row.append(lift_score)
    lifts.append(row)

cols = ['beer_name']
for attribute in attributes:
    cols.append(attribute)

lifts = pd.DataFrame(lifts, columns=cols)
lifts = lifts.set_index('beer_name')
lifts

,hoppy,malty,dark,complex
beer_name,,,,
King Sue,1.700400,0.980775,0.081186,0.642162
Heady Topper,3.274300,0.895490,0.186251,0.760206
KBS - Maple Mackinac Fudge,0.061779,0.948166,2.005782,0.627997
CBS (Canadian Breakfast Stout),0.129998,0.859028,1.733476,1.286677
KBS,0.000000,1.025685,1.821513,1.362886
Zombie Dust,3.534411,2.071853,0.204913,0.630314
Pseudo Sue,2.504119,1.367817,0.072590,0.502397
Pliny The Elder,3.146384,1.447275,0.115211,0.398687
Bourbon County Brand Stout,0.000000,0.684898,2.196162,1.899956


In [221]:
similar_beers = []
liked_beer = 'King Sue'
for beer in list(lifts.index):
    v1 = np.array(lifts.loc[liked_beer].values)
    v1 = v1.reshape(1, -1)
    if beer != liked_beer:
        v2 = np.array(lifts.loc[beer].values)
        v2 = v2.reshape(1, -1)
        cosines_sim = cosine_similarity(v1, v2)[0][0]
        similar_beers.append([beer, cosines_sim])

similar_beers = pd.DataFrame(similar_beers,columns=['beer_name', 'similarity'])
similar_beers = similar_beers.sort_values('similarity', ascending=False)
similar_beers

,beer_name,similarity
5,Pseudo Sue,0.989610
8,Focal Banger,0.989124
4,Zombie Dust,0.986614
6,Pliny The Elder,0.975772
0,Heady Topper,0.965110
2,CBS (Canadian Breakfast Stout),0.422158
3,KBS,0.393378
1,KBS - Maple Mackinac Fudge,0.335831
7,Bourbon County Brand Stout,0.335676
